In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

**<font size="5" color="red">ch4. RNN(Recurrent Neural Network 순환신경망</font>**
# 1. 문맥을 이용하여 모델만들기

In [2]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [3]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]
print(encoded)
print(t.word_index)

[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [8]:
# 문자열 리스트을 인덱스 시퀀스로 반환
print(t.texts_to_sequences(['경마장에 말이 있다','말이 뛰다']))
print(t.texts_to_sequences(['경마장에 말이 있다'])[0])

[[2, 1, 5], [1]]
[2, 1, 5]


In [11]:
print(t.texts_to_matrix(['말이 뛰고']))
print(t.texts_to_sequences(['말이 뛰고']))

[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[1, 4]]


In [ ]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [23]:
print(t.word_index)
for word, value in t.word_index.items():
    if value==2:
        print("2:", word)
        break

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}
2: 경마장에


In [18]:
# text를 학습시키기 위해 ['경마장에 있는', '경마장에 있는 말이', '경마장에 있는 말이 뛰고',...]
sequences = []
for line in text.split('\n'):
    print('원래 문장 :', line)
    encoded = t.texts_to_sequences([line])[0]
    print('encoded된 문장 :', encoded)
    for i in range(0, len(encoded)-1): # 시작 index(빨간색 화살표)
        for j in range(i+2, len(encoded)+1): # 끝나는 index(파란색 화살표)
            sequences.append(encoded[i:j])
# sequences
print('sequences와 해석을 출력')

원래 문장 : 경마장에 있는 말이 뛰고 있다
encoded된 문장 : [2, 3, 1, 4, 5]
원래 문장 : 그의 말이 법이다
encoded된 문장 : [6, 1, 7]
원래 문장 : 가는 말이 고와야 오는 말이 곱다
encoded된 문장 : [8, 1, 9, 10, 1, 11]


[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [3, 1],
 [3, 1, 4],
 [3, 1, 4, 5],
 [1, 4],
 [1, 4, 5],
 [4, 5],
 [6, 1],
 [6, 1, 7],
 [1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11],
 [1, 9],
 [1, 9, 10],
 [1, 9, 10, 1],
 [1, 9, 10, 1, 11],
 [9, 10],
 [9, 10, 1],
 [9, 10, 1, 11],
 [10, 1],
 [10, 1, 11],
 [1, 11]]

In [17]:
list(range(0,5+1))

[0, 1, 2, 3, 4, 5]